In [9]:
import geopandas as gpd
import pandas as pd

## Open State data

In [10]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Language 

In [11]:
language_df = pd.read_csv("data/ACSST1Y2023.S1601-2025-06-22T195612.csv", thousands=",")

In [12]:
language_df = language_df.T.reset_index()
language_df.columns = language_df.iloc[0].str.rstrip().str.lstrip()
language_df = language_df.loc[1:]

In [13]:
language_percent_df = language_df[
    language_df["Label (Grouping)"].str.split("!!").str[1] == "Percent"
]
language_percent_df["NAME"] = (
    language_percent_df["Label (Grouping)"].str.split("!!").str[0]
)

/tmp/ipykernel_32183/3217474478.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  language_percent_df["NAME"] = (


In [14]:
language_percent_df["ONLY_ENGLISH"] = (
    language_percent_df["Speak only English"]
    .iloc[:, 0]
    .str.replace("%", "")
    .astype(float)
)
language_percent_df = language_percent_df[["NAME", "ONLY_ENGLISH"]]

/tmp/ipykernel_32183/1095888668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  language_percent_df['ONLY_ENGLISH'] = language_percent_df['Speak only English'].iloc[:,0].str.replace("%","").astype(float)


## Merge Data

In [15]:
language_percent_gdf = states_df.merge(language_percent_df, on="NAME", how="left")

In [16]:
language_percent_gdf = language_percent_gdf.dropna()

In [17]:
language_percent_gdf = language_percent_gdf.to_crs(9311)
language_percent_gdf.to_file("data/language.gpkg")